<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Patches_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

Using TensorFlow backend.


In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = None, pooling = 'avg')
x = base_model.output
preds = Dense(3, activation='softmax', kernel_regularizer=l2(0.01))(x)
model = Model(inputs=base_model.input, outputs=preds)
top_layer_nb = 15
pos_cls_weight = 1.
neg_cls_weight = 1.
class_weight = {0:1.0, 1:pos_cls_weight, 2:neg_cls_weight} 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 2s 0us/step


In [0]:
org_model = model

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Patch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

#ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, partience = 2, verbose = 1)

#setting the learning rate
Adam = Adam(learning_rate=0.001)
SGD = SGD(learning_rate=0.001)

# First Stage Of training with learning rate 0.001

In [0]:
for layer in org_model.layers[:-1]:
  layer.trainable = False
#class_weight = { 0:1.0, 1:pos_cls_weight, 2:neg_cls_weight }
nb_worker = 1
if nb_worker == 1:
  pickle_safe = False
else:
  pickle_safe = True


In [0]:
train_data_dir = "/content/drive/My Drive/patches_sort"

In [10]:
datagen = ImageDataGenerator(validation_split=0.3)
batch_size = 32
img_width , img_height = 227, 227

# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 75530 images belonging to 3 classes.
Found 32368 images belonging to 3 classes.


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam,
              metrics=['accuracy'],
              )
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

Run the cell below for about 3 to 4 epoch

In [0]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=50,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1,
        class_weight = class_weight)

Epoch 1/50
32/32 [==============================] - 5s 166ms/step - loss: 1.1319 - acc: 0.4385

Epoch 00001: val_acc improved from 0.42676 to 0.43848, saving model to Patch_CNN.h5
32/32 [==============================] - 436s 14s/step - loss: 1.1569 - acc: 0.3926 - val_loss: 1.1319 - val_acc: 0.4385
Epoch 2/50
32/32 [==============================] - 5s 168ms/step - loss: 1.1078 - acc: 0.4561

Epoch 00002: val_acc improved from 0.43848 to 0.45605, saving model to Patch_CNN.h5
32/32 [==============================] - 337s 11s/step - loss: 1.1479 - acc: 0.4082 - val_loss: 1.1078 - val_acc: 0.4561
Epoch 3/50
32/32 [==============================] - 5s 166ms/step - loss: 1.1220 - acc: 0.4648

Epoch 00003: val_acc improved from 0.45605 to 0.46484, saving model to Patch_CNN.h5
32/32 [==============================] - 429s 13s/step - loss: 1.1175 - acc: 0.4297 - val_loss: 1.1220 - val_acc: 0.4648
Epoch 4/50
32/32 [==============================] - 5s 172ms/step - loss: 1.1080 - acc: 0.4619

E

In [0]:
loss_history = hist.history['val_loss']
acc_history = hist.history['val_acc']

# Second Stage of training with learning rate of 0.0001

In [0]:
Adam1 = tf.keras.optimizers.Adam(learning_rate = 0.0001)
for layer in org_model.layers[top_layer_nb:]:
  layer.trainable = True
model.compile(loss='categorical_crossentropy',
              optimizer=Adam1,
              metrics=['accuracy'],
              )
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer1 = ModelCheckpoint(filepath='2ndPatch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping1 = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

In [0]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=len(loss_history)+10,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer1, early_stopping1],
        verbose=1,
        initial_epoch = len(loss_history),
        class_weight = class_weight)

Epoch 4/13
32/32 [==============================] - 7s 218ms/step - loss: 1.1065 - acc: 0.3955

Epoch 00004: val_acc improved from -inf to 0.39551, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 585s 18s/step - loss: 1.2802 - acc: 0.3662 - val_loss: 1.1065 - val_acc: 0.3955
Epoch 5/13
32/32 [==============================] - 7s 218ms/step - loss: 1.1223 - acc: 0.4062

Epoch 00005: val_acc improved from 0.39551 to 0.40625, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 626s 20s/step - loss: 1.1110 - acc: 0.4131 - val_loss: 1.1223 - val_acc: 0.4062
Epoch 6/13
32/32 [==============================] - 7s 217ms/step - loss: 1.1766 - acc: 0.4463

Epoch 00006: val_acc improved from 0.40625 to 0.44629, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 605s 19s/step - loss: 1.1058 - acc: 0.3945 - val_loss: 1.1766 - val_acc: 0.4463
Epoch 7/13
32/32 [==============================] - 7s 220ms/step - loss: 1.1813 - acc: 0.4

In [0]:
import numpy as np
loss_history = np.append(loss_history, hist.history['val_loss'])
acc_history = np.append(acc_history, hist.history['val_acc'])

# Third Stage of training with learning rate 0.00001

In [0]:
for layers in org_model.layers[:top_layer_nb]:
  layer.trainable = True

In [0]:
Adam2 = tf.keras.optimizers.Adam(learning_rate = 0.00001)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam2,
              metrics=['accuracy'],
              )
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
mod = tf.keras.models.load_model("/content/Patch_CNN.h5")
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer1 = ModelCheckpoint(filepath='3rdPatch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping1 = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=len(loss_history)+37,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer1, early_stopping1],
        verbose=1,
        initial_epoch = len(loss_history),
        class_weight = class_weight)

Epoch 14/50
32/32 [==============================] - 7s 225ms/step - loss: 1.8808 - acc: 0.4834

Epoch 00014: val_acc improved from -inf to 0.48340, saving model to 3rdPatch_CNN.h5
32/32 [==============================] - 411s 13s/step - loss: 1.0363 - acc: 0.4326 - val_loss: 1.8808 - val_acc: 0.4834
Epoch 15/50
32/32 [==============================] - 7s 225ms/step - loss: 2.0609 - acc: 0.4775

Epoch 00015: val_acc did not improve from 0.48340
32/32 [==============================] - 401s 13s/step - loss: 1.0175 - acc: 0.4492 - val_loss: 2.0609 - val_acc: 0.4775
Epoch 16/50
32/32 [==============================] - 7s 228ms/step - loss: 2.0570 - acc: 0.4795

Epoch 00016: val_acc did not improve from 0.48340
32/32 [==============================] - 555s 17s/step - loss: 1.0372 - acc: 0.4600 - val_loss: 2.0570 - val_acc: 0.4795
Epoch 17/50
32/32 [==============================] - 7s 229ms/step - loss: 2.1962 - acc: 0.4980

Epoch 00017: val_acc improved from 0.48340 to 0.49805, saving mod